In [1]:
import pandas as pd
import numpy as np

In [2]:
# read in data
util_df = pd.read_csv(r'C:\Users\Erik\Downloads\Utilization Tabular (12).csv', sep='\t',
                       encoding='utf_16_le')
util_df['Hours Date'] = pd.to_datetime(util_df['Hours Date'])
filt = util_df['Hours Date'] < pd.to_datetime('2020-10-31')
util_df = util_df[filt]
org_df = pd.read_csv(r'C:\Users\Erik\Downloads\Organizations.csv', sep='\t',
                       encoding='utf_16_le')
emp_df = pd.read_csv(r'C:\Users\Erik\Downloads\Employees (14).csv', sep='\t',
                       encoding='utf_16_le')
emp_df['Hire date'] = pd.to_datetime(emp_df['Hire date'])
emp_df['Termination date'] = pd.to_datetime(emp_df['Termination date'])

In [3]:
util_df.head()

,Employee ID,Last Name,First Name,Work Schedule Description,Org Name,Project ID,Project Name,User Defined Code 3,Hours Date,Entered Hours,Approved Hours,Comments
0,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.001.009.01,Planning and Ops CH,G&A,2020-06-01,10.63,0,NaN
1,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.005.001.01,R&D General Intl,IRD,2020-06-01,0.58,0,NaN
2,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.019.009.01,COVID-19 CH,G&A,2020-06-01,0.92,0,NaN
3,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.001.001.01,Planning and Ops Gen Intl,OVH,2020-06-02,1.50,0,NaN
4,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.001.009.01,Planning and Ops CH,G&A,2020-06-02,7.50,0,NaN


In [4]:
org_df.head()

,Project ID,Project Name,Organization ID,Organization Name,Level Number
0,1001,USAID Measuring Impact II,1.01.01.01,General Intl,4
1,1001.AFR,BI-AFR,1.01.01.04,Africa,4
2,1001.AFR.001,BI-AFR,1.01.01.04,Africa,4
3,1001.AFR.001.01,16.0.AFR_BuyIn_Mgmt,1.01.01.04,Africa,4
4,1001.AFR.001.02,16.0.AFR_Zambia FS,1.01.01.04,Africa,4


In [5]:
# merge hours entries and organizations
df = pd.merge(util_df, org_df, how='left')
df.head()

,Employee ID,Last Name,First Name,Work Schedule Description,Org Name,Project ID,Project Name,User Defined Code 3,Hours Date,Entered Hours,Approved Hours,Comments,Organization ID,Organization Name,Level Number
0,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.001.009.01,Planning and Ops CH,G&A,2020-06-01,10.63,0,NaN,1.01.90.01,HR and Operations,4.0
1,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.005.001.01,R&D General Intl,IRD,2020-06-01,0.58,0,NaN,1.01.01.01,General Intl,4.0
2,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.019.009.01,COVID-19 CH,G&A,2020-06-01,0.92,0,NaN,1.01.90.01,HR and Operations,4.0
3,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.001.001.01,Planning and Ops Gen Intl,OVH,2020-06-02,1.50,0,NaN,1.01.01.01,General Intl,4.0
4,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.001.009.01,Planning and Ops CH,G&A,2020-06-02,7.50,0,NaN,1.01.90.01,HR and Operations,4.0


In [6]:
# confirm merge did not add new time entries
len(util_df) - len(df)

0

In [7]:
df.columns

Index(['Employee ID', 'Last Name', 'First Name', 'Work Schedule Description',
       'Org Name', 'Project ID', 'Project Name', 'User Defined Code 3',
       'Hours Date', 'Entered Hours', 'Approved Hours', 'Comments',
       'Organization ID', 'Organization Name', 'Level Number'],
      dtype='object')

In [8]:
df['Organization Name'].unique()

array(['HR and Operations', 'General Intl', 'Global Adaptive Managemen',
       'Latin America & the Carib', 'General Domestic',
       'Environmental Incentives', 'Comms & KM', 'HR & Ops General',
       'Contract&Fin Spec Initiat', 'HR & Ops Special Initiat', 'Water',
       'Africa', nan, 'Habitat', 'Marketing General'], dtype=object)

In [9]:
emp_df.head()

,Employee ID,Last Name,First Name,Middle Initial,E-mail Address,Active Flag,Hire date,Termination date,Work Schedule,Work Schedule Description,Default Org
0,100041,Abragan,Maria Celes,L,mabragan@enviroincentives.com,Y,2019-04-08,NaT,STD,Standard,1.01.01
1,100003,Ajroud,Brittany,N,bajroud@enviroincentives.com,Y,2016-10-18,NaT,STD,Standard,1.01.01
2,100001,Alexandrovich,Andrew,NaN,andrew@enviroincentives.com,Y,2010-04-05,NaT,STD,Standard,1.01.90
3,100022,Anderson,Erik,T,eanderson@enviroincentives.com,Y,2014-03-17,NaT,STD,Standard,1.01.02
4,100027,Armanino,Molly,NaN,marmanino@enviroincentives.com,N,2017-06-18,2019-12-14,NaN,NaN,NaN


In [10]:
# fte hours is total number of fte days by start and end dates (ignore part time and very part time (e.g., CB))
sem_start = pd.to_datetime('2020-04-01')
sem_end = pd.to_datetime('2020-10-31')

fte_df = emp_df[['Employee ID', 'Hire date', 'Termination date']].copy()

def update_start(hire_date):
    if hire_date < sem_start:
        return sem_start
    elif hire_date > sem_end:
        return sem_end
    else:
        return hire_date
    
def update_end(termination_date):
    if pd.isnull(termination_date):
        return sem_end
    if termination_date > sem_end:
        return sem_end
    elif termination_date < sem_start:
        return sem_start
    else:
        return termination_date

fte_df['sem_start'] = fte_df['Hire date'].apply(update_start)
fte_df['sem_end'] = fte_df['Termination date'].apply(update_end)
fte_df['bushrs'] = np.busday_count(fte_df['sem_start'].dt.date, fte_df['sem_end'].dt.date) * 8

fte_df.head()

,Employee ID,Hire date,Termination date,sem_start,sem_end,bushrs
0,100041,2019-04-08,NaT,2020-04-01,2020-10-31,1224
1,100003,2016-10-18,NaT,2020-04-01,2020-10-31,1224
2,100001,2010-04-05,NaT,2020-04-01,2020-10-31,1224
3,100022,2014-03-17,NaT,2020-04-01,2020-10-31,1224
4,100027,2017-06-18,2019-12-14,2020-04-01,2020-04-01,0


In [11]:
fte_hrs = fte_df['bushrs'].sum()
fte_hrs

64760

In [12]:
# total hours is straight sum 
total_hrs = df['Entered Hours'].sum()
total_hrs

61166.06

In [13]:
# the proportion of total hours worked is slightly less than full time, consistent with some employees being part time
total_hrs/fte_hrs

0.9445037059913527

In [14]:
# review fte by employee based on expected fte hours as calculated
hrs_by_emp = df.groupby('Employee ID').sum()['Entered Hours']
hrs_by_emp = pd.merge(hrs_by_emp, fte_df, left_index=True, right_on='Employee ID', how='left')
hrs_by_emp['fte'] = hrs_by_emp['Entered Hours'] / hrs_by_emp['bushrs']
hrs_by_emp = pd.merge(hrs_by_emp, emp_df, on='Employee ID')
hrs_by_emp.loc[:, ['Last Name', 'First Name', 'Entered Hours', 'sem_start', 'sem_end', 'bushrs', 'fte']].sort_values('fte', ascending=False)

,Last Name,First Name,Entered Hours,sem_start,sem_end,bushrs,fte
64,Byenkya,Tina,296.00,2020-10-02,2020-10-31,168,1.761905
62,Owusu,Philip,364.00,2020-09-15,2020-10-31,272,1.338235
63,Walter,Karen,286.50,2020-09-22,2020-10-31,232,1.234914
13,Hoye,Susan,1506.25,2020-04-01,2020-10-31,1224,1.230596
34,Shay,Arica,1449.32,2020-04-01,2020-10-31,1224,1.184085
...,...,...,...,...,...,...,...
6,Gambrill,Amy,1148.67,2020-04-01,2020-10-31,1224,0.938456
5,Flower,Kathleen,1000.07,2020-04-01,2020-10-31,1224,0.817051
27,Brock,Cameryn,625.98,2020-06-15,2020-10-31,800,0.782475
4,Dubois,Natalie,914.25,2020-04-01,2020-10-31,1224,0.746936


In [15]:
# what is happening with high ftes for some new employees?
filt = df['Last Name'].isin(['Byenkya'])
df[filt]

,Employee ID,Last Name,First Name,Work Schedule Description,Org Name,Project ID,Project Name,User Defined Code 3,Hours Date,Entered Hours,Approved Hours,Comments,Organization ID,Organization Name,Level Number
22222,100097.0,Byenkya,Tina,Standard Kenya,Africa,1009.003.202.01,Somalia Program,SRV,2020-10-02,16.0,0,Adding time after getting access to timesheet,NaN,NaN,NaN
22223,100097.0,Byenkya,Tina,Standard Kenya,Africa,1009.003.202.01,Somalia Program,SRV,2020-10-05,16.0,0,Adding time after getting access to timesheet,NaN,NaN,NaN
22224,100097.0,Byenkya,Tina,Standard Kenya,Africa,1009.003.202.01,Somalia Program,SRV,2020-10-06,16.0,0,Adding time after getting access to timesheet,NaN,NaN,NaN
22225,100097.0,Byenkya,Tina,Standard Kenya,Africa,1009.003.202.01,Somalia Program,SRV,2020-10-07,16.0,0,Adding time after getting access to timesheet,NaN,NaN,NaN
22226,100097.0,Byenkya,Tina,Standard Kenya,Africa,1009.003.202.01,Somalia Program,SRV,2020-10-08,16.0,0,Adding time after getting access to timesheet,NaN,NaN,NaN
22227,100097.0,Byenkya,Tina,Standard Kenya,Africa,1009.003.202.01,Somalia Program,SRV,2020-10-09,16.0,0,Adding time after getting access to timesheet,NaN,NaN,NaN
22228,100097.0,Byenkya,Tina,Standard Kenya,Africa,1009.003.202.01,Somalia Program,SRV,2020-10-12,8.0,0,Adding time after getting access to timesheet,NaN,NaN,NaN
22229,100097.0,Byenkya,Tina,Standard Kenya,Africa,1009.003.202.01,Somalia Program,SRV,2020-10-12,8.0,0,I worked on this day. Based on guidance from A...,NaN,NaN,NaN
22230,100097.0,Byenkya,Tina,Standard Kenya,Africa,1009.003.202.01,Somalia Program,SRV,2020-10-13,16.0,0,Adding time after getting access to timesheet,NaN,NaN,NaN
22231,100097.0,Byenkya,Tina,Standard Kenya,Africa,1009.003.202.01,Somalia Program,SRV,2020-10-14,16.0,0,Adding time after getting access to timesheet,NaN,NaN,NaN


In [16]:
# drop Byenkya due to irregularities in data entries (too many 16 hour days)
df = df[~filt]
df

,Employee ID,Last Name,First Name,Work Schedule Description,Org Name,Project ID,Project Name,User Defined Code 3,Hours Date,Entered Hours,Approved Hours,Comments,Organization ID,Organization Name,Level Number
0,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.001.009.01,Planning and Ops CH,G&A,2020-06-01,10.63,0,NaN,1.01.90.01,HR and Operations,4.0
1,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.005.001.01,R&D General Intl,IRD,2020-06-01,0.58,0,NaN,1.01.01.01,General Intl,4.0
2,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.019.009.01,COVID-19 CH,G&A,2020-06-01,0.92,0,NaN,1.01.90.01,HR and Operations,4.0
3,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.001.001.01,Planning and Ops Gen Intl,OVH,2020-06-02,1.50,0,NaN,1.01.01.01,General Intl,4.0
4,100001.0,Alexandrovich,Andrew,Standard,Company Health,INDR.001.009.01,Planning and Ops CH,G&A,2020-06-02,7.50,0,NaN,1.01.90.01,HR and Operations,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29661,100073.0,Wolf,Laurel,Standard,International,NaN,SWS Task 3a: Data Management,SRV,2020-05-28,0.50,0,NaN,NaN,NaN,NaN
29662,100073.0,Wolf,Laurel,Standard,International,NaN,LAC ESSC,SRV,2020-05-29,5.00,0,NaN,NaN,NaN,NaN
29663,100073.0,Wolf,Laurel,Standard,International,NaN,Planning and Ops Int'l,G&A,2020-05-29,1.00,0,NaN,NaN,NaN,NaN
29664,100073.0,Wolf,Laurel,Standard,International,NaN,SWS Task 3a: Data Management,SRV,2020-05-29,1.00,0,NaN,NaN,NaN,NaN


In [17]:
# calculate total hours by organization
org_total_hrs = df.groupby('Organization Name').sum()['Entered Hours']
org_total_hrs

Organization Name
Africa                        2145.77
Comms & KM                    4675.21
Contract&Fin Spec Initiat       17.25
Environmental Incentives      4348.32
General Domestic               145.47
General Intl                  3687.80
Global Adaptive Managemen    11862.43
HR & Ops General               607.80
HR & Ops Special Initiat       538.68
HR and Operations             4072.71
Habitat                       3665.02
Latin America & the Carib     6390.91
Marketing General              342.72
Water                         3308.03
Name: Entered Hours, dtype: float64

In [18]:
# calculate billable hours by organization
filt = df['User Defined Code 3'] == 'SRV'
org_bill_hrs = df.loc[filt].groupby('Organization Name').sum()['Entered Hours']
org_bill_hrs

Organization Name
Africa                        1559.00
Comms & KM                    3183.09
General Intl                   285.14
Global Adaptive Managemen    10412.95
Habitat                       2470.03
Latin America & the Carib     5689.32
Water                         1866.70
Name: Entered Hours, dtype: float64

In [19]:
# join to single df
hrs_df = pd.merge(org_bill_hrs, org_total_hrs, left_index=True, right_index=True, suffixes=('_bill', '_total'))
hrs_df

,Entered Hours_bill,Entered Hours_total
Organization Name,,
Africa,1559.00,2145.77
Comms & KM,3183.09,4675.21
General Intl,285.14,3687.80
Global Adaptive Managemen,10412.95,11862.43
Habitat,2470.03,3665.02
Latin America & the Carib,5689.32,6390.91
Water,1866.70,3308.03


In [20]:
# divide org total hrs by total hours to get proportion
hrs_df['prop_to_org'] = hrs_df['Entered Hours_total'] / total_hrs
hrs_df

,Entered Hours_bill,Entered Hours_total,prop_to_org
Organization Name,,,
Africa,1559.00,2145.77,0.035081
Comms & KM,3183.09,4675.21,0.076435
General Intl,285.14,3687.80,0.060292
Global Adaptive Managemen,10412.95,11862.43,0.193938
Habitat,2470.03,3665.02,0.059919
Latin America & the Carib,5689.32,6390.91,0.104485
Water,1866.70,3308.03,0.054083


In [22]:
# weight fte by prop to org
hrs_df['weighted_fte'] = hrs_df['prop_to_org'] * fte_hrs
hrs_df

,Entered Hours_bill,Entered Hours_total,prop_to_org,weighted_fte
Organization Name,,,,
Africa,1559.00,2145.77,0.035081,2271.849212
Comms & KM,3183.09,4675.21,0.076435,4949.911758
General Intl,285.14,3687.80,0.060292,3904.484415
Global Adaptive Managemen,10412.95,11862.43,0.193938,12559.431927
Habitat,2470.03,3665.02,0.059919,3880.365928
Latin America & the Carib,5689.32,6390.91,0.104485,6766.421306
Water,1866.70,3308.03,0.054083,3502.400233


In [23]:
# utilization is billale hours divided by weighted fte
hrs_df['utilization'] = hrs_df['Entered Hours_bill'] / hrs_df['weighted_fte']
hrs_df.to_csv(r'C:\Users\Erik\Downloads\portfolio_utilization.csv')
hrs_df

,Entered Hours_bill,Entered Hours_total,prop_to_org,weighted_fte,utilization
Organization Name,,,,,
Africa,1559.00,2145.77,0.035081,2271.849212,0.686225
Comms & KM,3183.09,4675.21,0.076435,4949.911758,0.643060
General Intl,285.14,3687.80,0.060292,3904.484415,0.073029
Global Adaptive Managemen,10412.95,11862.43,0.193938,12559.431927,0.829094
Habitat,2470.03,3665.02,0.059919,3880.365928,0.636546
Latin America & the Carib,5689.32,6390.91,0.104485,6766.421306,0.840817
Water,1866.70,3308.03,0.054083,3502.400233,0.532977


In [29]:
# what proportion of hours went towards billale projects
hrs_df['Entered Hours_bill'].sum() / total_hrs

0.4163457643013135

In [30]:
hrs_df['Entered Hours_bill'].sum() / fte_hrs

0.39324011735639275

In [25]:
# Note that Molly does not appear to be updated in the system
filt = emp_df['Last Name'] == 'Armanino'
emp_df.loc[filt]

,Employee ID,Last Name,First Name,Middle Initial,E-mail Address,Active Flag,Hire date,Termination date,Work Schedule,Work Schedule Description,Default Org
4,100027,Armanino,Molly,NaN,marmanino@enviroincentives.com,N,2017-06-18,2019-12-14,NaN,NaN,NaN


In [26]:
# However she also does not appear to be billing
filt = util_df['Last Name'] == 'Armanino'
util_df.loc[filt]

,Employee ID,Last Name,First Name,Work Schedule Description,Org Name,Project ID,Project Name,User Defined Code 3,Hours Date,Entered Hours,Approved Hours,Comments


In [27]:
util_df['Last Name'].unique()

array(['Alexandrovich', 'Ajroud', 'Chandrasekaran', 'Cook', 'Dubois',
       'Flower', 'Gambrill', 'Gibert', 'Hardeman', 'Hicks', 'Hoye',
       'Lauck', 'Nease', 'Peabody', 'Present', 'Schueler', 'Anderson',
       'Boysen', 'Motlow', 'Riley', 'Sokulsky', 'Praul', 'Hansen',
       'Guetschow', 'Brock', 'Chesterman', 'Abragan', 'Uhl', 'Exline',
       'King', 'Chery', 'Shay', 'Baca', 'Martinez-Sanchez', 'Grange',
       'Castillo Ferri', 'Nico', 'Wolf', 'Mirghani', 'Motolinia',
       'Daniels', 'Durand', 'Haik', 'Bevins', 'Wong', 'Sarkisian',
       'Spencer', 'Ballard', 'Boutemy', 'Schmidt', 'Giannoni', 'Tripp',
       'Reilly', 'Witz', 'Fong', 'Connolly', 'Masood', 'Peimbert',
       'Owusu', 'Walter', 'Byenkya', 'Richards', 'Myers', 'Kapikinyu',
       'Stiger'], dtype=object)